In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from sklearn.datasets import fetch_olivetti_faces
olivetti = fetch_olivetti_faces()
pretrain_x=olivetti.data
pretrain_y=olivetti.target

In [0]:
pretrain_x.shape

(400, 4096)

In [0]:
pretrain_y.shape

(400,)

In [0]:
pretrain_y = pretrain_y==20

In [0]:
pretrain_y.shape

(400,)

In [0]:
pretrain_y.sum()

10

In [0]:
def f1(y_true, y_pred):
    y_pred = np.rint(y_pred)
    tp = sum(y_true*y_pred)
    tn = sum((1-y_true)*(1-y_pred))
    fp = sum((1-y_true)*y_pred)
    fn = sum(y_true*(1-y_pred))

    p = tp / (tp + fp )
    r = tp / (tp + fn )

    f1 = 2*p*r / (p+r)
    return f1

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.014233112335205078 seconds ---


In [0]:
start_time = time.time()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.790295124053955 seconds ---


In [0]:
X_train, X_test, y_williams_train, y_williams_test = train_test_split(X,y_williams, test_size=1./3, random_state = 6926, shuffle = True, stratify = y_williams)

In [0]:
X_train = X_train.reshape(8822,64,64,1)

In [0]:
X_test = X_test.reshape(4411,64,64,1)

In [0]:
px_1,px_2,py_1,py_2  = train_test_split(pretrain_x,pretrain_y, test_size=1./3, random_state = 6926, shuffle = True, stratify = pretrain_y)

In [0]:
px_1.shape

(266, 4096)

In [0]:
px_1 =px_1.reshape(266,64,64,1)

In [0]:
px_2 = px_2.reshape(134,64,64,1)

In [0]:
williams_model = Sequential()
williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Flatten())
williams_model.add(Dense(1000,activation='relu'))
williams_model.add(Dense(1,activation='sigmoid'))
williams_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training.log')
williams_model.fit(px_1,py_1, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])

Epoch 1/10
266/266 [==============================] - 1s 3ms/step - loss: 0.6965 - acc: 0.0263
Epoch 2/10
266/266 [==============================] - 0s 453us/step - loss: 0.6405 - acc: 0.9737
Epoch 3/10
266/266 [==============================] - 0s 429us/step - loss: 0.5003 - acc: 0.9737
Epoch 4/10
266/266 [==============================] - 0s 413us/step - loss: 0.2629 - acc: 0.9737
Epoch 5/10
266/266 [==============================] - 0s 390us/step - loss: 0.1181 - acc: 0.9737
Epoch 6/10
266/266 [==============================] - 0s 385us/step - loss: 0.1611 - acc: 0.9737
Epoch 7/10
266/266 [==============================] - 0s 386us/step - loss: 0.2214 - acc: 0.9737
Epoch 8/10
266/266 [==============================] - 0s 360us/step - loss: 0.2382 - acc: 0.9737
Epoch 9/10
266/266 [==============================] - 0s 366us/step - loss: 0.2180 - acc: 0.9737
Epoch 10/10
266/266 [==============================] - 0s 370us/step - loss: 0.1804 - acc: 0.9737


In [0]:
print(f1_score(py_1,williams_model.predict_classes(px_1)))
print(f1_score(py_2,williams_model.predict_classes(px_2)))

0.0
0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
williams_model.save("/content/gdrive/My Drive/data/init_w.model")

In [0]:
williams_model.fit(X_train, y_williams_train, batch_size = 500, epochs = 30, verbose =1 ,callbacks=[csv_logger])
print(f1_score(y_williams_train,williams_model.predict_classes(X_train)))
print(f1_score(y_williams_test,williams_model.predict_classes(X_test)))

Epoch 1/30
8822/8822 [==============================] - 3s 361us/step - loss: 0.0325 - acc: 0.9960
Epoch 2/30
8822/8822 [==============================] - 3s 359us/step - loss: 0.0243 - acc: 0.9960
Epoch 3/30
8822/8822 [==============================] - 3s 360us/step - loss: 0.0244 - acc: 0.9960
Epoch 4/30
8822/8822 [==============================] - 3s 361us/step - loss: 0.0237 - acc: 0.9960
Epoch 5/30
8822/8822 [==============================] - 3s 358us/step - loss: 0.0229 - acc: 0.9960
Epoch 6/30
8822/8822 [==============================] - 3s 358us/step - loss: 0.0234 - acc: 0.9960
Epoch 7/30
8822/8822 [==============================] - 3s 360us/step - loss: 0.0220 - acc: 0.9960
Epoch 8/30
8822/8822 [==============================] - 3s 360us/step - loss: 0.0215 - acc: 0.9960
Epoch 9/30
8822/8822 [==============================] - 3s 362us/step - loss: 0.0217 - acc: 0.9960
Epoch 10/30
8822/8822 [==============================] - 3s 361us/step - loss: 0.0187 - acc: 0.9960
Epoch 11/

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams.model")